In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import networkx as nx
import random
import pickle
import itertools

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
from tensorflow.linalg import diag
from func_timeout import func_timeout, FunctionTimedOut
from tensorflow.keras import callbacks
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import normalized_mutual_info_score

In [2]:
def generate_network(num_vertices, avg_degree):
    mu = random.uniform(0.2, 0.4)
    return nx.LFR_benchmark_graph(num_vertices, 2.5, 1.5, mu, avg_degree, max_iters=100)

In [3]:
def create_adjacency_matrix(graph):
    return np.asarray(nx.adjacency_matrix(graph).todense())

def create_pairwise_community_indicator_matrix(community_list):
    num_vertices = len(community_list)
    H = np.zeros(shape=(num_vertices, num_vertices))
    for community in community_list:
        for pair in itertools.combinations_with_replacement(community_list, 2):
            H[pair[0],pair[1]] = 1
    return H

def create_laplacian_matrix(ci_matrix):
    return tf.linalg.diag(tf.reduce_sum(ci_matrix, 1)) - ci_matrix

def combine_data(B, L):
    return np.concatenate((B,L), axis=1)

def get_community_list(graph):
    com_list = list({frozenset(graph.nodes[v]['community']) for v in graph})
    communities = [list(com_list[x]) for x in range(0, len(com_list))]
    return communities
    
def get_community_labels(graph):
    communities = get_community_list(graph)
    labels = np.zeros(len(graph), dtype=int)
    x = 0
    for community in communities:
        for vertex in community:
            labels[vertex] = x
        x += 1
    return labels

def reconstruct_communities(labels):
    communities = {}
    for x in range(max(labels)+1):
        communities[x] = []
    i = 0
    for x in labels:
        communities[x].append(i)
        i += 1
    return communities

def karate_club_communities(graph):
    clublist = [(graph.nodes[v]['club']) for v in graph]
    communities = np.zeros(len(graph), dtype=int)
    x = 0
    for el in clublist:
        if el == 'Officer':
            communities[x] = 1
        x += 1
    return communities

In [4]:
class Autoencoder(Model):
    
    def __init__(self, lda, n, latentdim):
        super(Autoencoder, self).__init__()
        self.lda = lda
        self.n = n
        self.encoder = tf.keras.Sequential([
            layers.Dense(latentdim, activation='relu')
        ])
        self.decoder = tf.keras.Sequential([
            layers.Dense(n, activation='sigmoid')
        ])
        
    def call(self, X):
        x = X[:,0:self.n]
        L = X[:,self.n:2*self.n]
        encoded = self.encoder(x)
        H = encoded
        #I can't get the pairwise constraints bit to work - the model doesn't converge with it and idk why
        #self.add_loss(self.lda * tf.linalg.trace(tf.linalg.matmul(tf.transpose(H), tf.linalg.matmul(L, H)))
        decoded = self.decoder(encoded)
        return decoded

In [5]:
#KARATE CLUB
kc = nx.karate_club_graph()

B_kc_1 = tf.convert_to_tensor(nx.modularity_matrix(kc))
O_kc = create_pairwise_community_indicator_matrix(karate_club_communities(kc))
L_kc = create_laplacian_matrix(O_kc)
#modularity matrix + pairwise community membership matrix
X_kc_1 = tf.convert_to_tensor(combine_data(B_kc_1, L_kc))

In [6]:
ae_kc_1 = Autoencoder(1000, 34, 32)
ae_kc_1.compile(optimizer='adam', loss="mse")
ae_kc_1.fit(X_kc_1, B_kc_1, epochs=1000, shuffle=True, batch_size=34)

Epoch 1/1000
1/1 [==============================] - 0s 413ms/step - loss: 0.3659
Epoch 2/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.3642
Epoch 3/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.3626
Epoch 4/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.3609
Epoch 5/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.3593
Epoch 6/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.3576
Epoch 7/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.3560
Epoch 8/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.3544
Epoch 9/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.3528
Epoch 10/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.3512
Epoch 11/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.3496
Epoch 12/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.3480
Epoch 13/1000
1/1 [============================

1/1 [==============================] - 0s 4ms/step - loss: 0.1866
Epoch 104/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.1848
Epoch 105/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.1830
Epoch 106/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.1813
Epoch 107/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.1796
Epoch 108/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.1779
Epoch 109/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.1762
Epoch 110/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.1746
Epoch 111/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.1729
Epoch 112/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.1713
Epoch 113/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.1697
Epoch 114/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.1681
Epoch 115/1000
1/1 [=======================

1/1 [==============================] - 0s 3ms/step - loss: 0.0976
Epoch 205/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0973
Epoch 206/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0970
Epoch 207/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0967
Epoch 208/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0964
Epoch 209/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.0961
Epoch 210/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.0958
Epoch 211/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.0955
Epoch 212/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0952
Epoch 213/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0949
Epoch 214/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0946
Epoch 215/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.0943
Epoch 216/1000
1/1 [=======================

1/1 [==============================] - 0s 3ms/step - loss: 0.0759
Epoch 306/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0757
Epoch 307/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0756
Epoch 308/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.0754
Epoch 309/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0752
Epoch 310/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0751
Epoch 311/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0749
Epoch 312/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0747
Epoch 313/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0746
Epoch 314/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.0744
Epoch 315/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.0742
Epoch 316/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0741
Epoch 317/1000
1/1 [=======================

1/1 [==============================] - 0s 4ms/step - loss: 0.0612
Epoch 407/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0611
Epoch 408/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0610
Epoch 409/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0609
Epoch 410/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0607
Epoch 411/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0606
Epoch 412/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0605
Epoch 413/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0604
Epoch 414/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0603
Epoch 415/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.0602
Epoch 416/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0600
Epoch 417/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0599
Epoch 418/1000
1/1 [=======================

1/1 [==============================] - 0s 2ms/step - loss: 0.0518
Epoch 508/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0517
Epoch 509/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0517
Epoch 510/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.0516
Epoch 511/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0515
Epoch 512/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.0515
Epoch 513/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0514
Epoch 514/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0513
Epoch 515/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.0512
Epoch 516/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0512
Epoch 517/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.0511
Epoch 518/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0510
Epoch 519/1000
1/1 [=======================

1/1 [==============================] - 0s 3ms/step - loss: 0.0458
Epoch 609/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.0458
Epoch 610/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.0457
Epoch 611/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.0457
Epoch 612/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0456
Epoch 613/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0456
Epoch 614/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0455
Epoch 615/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.0455
Epoch 616/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.0454
Epoch 617/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0454
Epoch 618/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0454
Epoch 619/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.0453
Epoch 620/1000
1/1 [=======================

1/1 [==============================] - 0s 2ms/step - loss: 0.0421
Epoch 710/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0421
Epoch 711/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.0421
Epoch 712/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0420
Epoch 713/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0420
Epoch 714/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0420
Epoch 715/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0420
Epoch 716/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.0419
Epoch 717/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0419
Epoch 718/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0419
Epoch 719/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.0418
Epoch 720/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0418
Epoch 721/1000
1/1 [=======================

1/1 [==============================] - 0s 4ms/step - loss: 0.0397
Epoch 811/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0396
Epoch 812/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0396
Epoch 813/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0396
Epoch 814/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0396
Epoch 815/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.0396
Epoch 816/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0395
Epoch 817/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0395
Epoch 818/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0395
Epoch 819/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0395
Epoch 820/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.0394
Epoch 821/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0394
Epoch 822/1000
1/1 [=======================

1/1 [==============================] - 0s 4ms/step - loss: 0.0377
Epoch 912/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0377
Epoch 913/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0377
Epoch 914/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.0377
Epoch 915/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0377
Epoch 916/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.0377
Epoch 917/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.0377
Epoch 918/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0376
Epoch 919/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.0376
Epoch 920/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0376
Epoch 921/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0376
Epoch 922/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.0376
Epoch 923/1000
1/1 [=======================

In [7]:
pickle.dump(kc, open("kc.p", "wb"))
ae_kc_1.save("ae_kc_1")

INFO:tensorflow:Assets written to: ae_kc_1\assets


In [8]:
M_kc_1 = ae_kc_1.predict(X_kc_1)
H_kc_1 = ae_kc_1.encoder(B_kc_1)
X_kc_2 = tf.convert_to_tensor(combine_data(H_kc_1, L_kc))

In [10]:
ae_kc_2 = Autoencoder(1000, 32, 16)
ae_kc_2.compile(optimizer='adam', loss="mse")
ae_kc_2.fit(X_kc_2, H_kc_1, epochs=3000, shuffle=True, batch_size=32)

Epoch 1/3000
2/2 [==============================] - 0s 3ms/step - loss: 0.8263
Epoch 2/3000
2/2 [==============================] - 0s 2ms/step - loss: 0.8224
Epoch 3/3000
2/2 [==============================] - 0s 3ms/step - loss: 0.8193
Epoch 4/3000
2/2 [==============================] - 0s 5ms/step - loss: 0.8164
Epoch 5/3000
2/2 [==============================] - 0s 4ms/step - loss: 0.8135
Epoch 6/3000
2/2 [==============================] - 0s 4ms/step - loss: 0.8106
Epoch 7/3000
2/2 [==============================] - 0s 6ms/step - loss: 0.8078
Epoch 8/3000
2/2 [==============================] - 0s 2ms/step - loss: 0.8052
Epoch 9/3000
2/2 [==============================] - 0s 3ms/step - loss: 0.8026
Epoch 10/3000
2/2 [==============================] - 0s 3ms/step - loss: 0.8001
Epoch 11/3000
2/2 [==============================] - 0s 3ms/step - loss: 0.7977
Epoch 12/3000
2/2 [==============================] - 0s 5ms/step - loss: 0.7953
Epoch 13/3000
2/2 [==============================

2/2 [==============================] - 0s 2ms/step - loss: 0.5860
Epoch 104/3000
2/2 [==============================] - 0s 2ms/step - loss: 0.5844
Epoch 105/3000
2/2 [==============================] - 0s 4ms/step - loss: 0.5829
Epoch 106/3000
2/2 [==============================] - 0s 998us/step - loss: 0.5814
Epoch 107/3000
2/2 [==============================] - 0s 2ms/step - loss: 0.5800
Epoch 108/3000
2/2 [==============================] - 0s 3ms/step - loss: 0.5787
Epoch 109/3000
2/2 [==============================] - 0s 3ms/step - loss: 0.5773
Epoch 110/3000
2/2 [==============================] - 0s 2ms/step - loss: 0.5759
Epoch 111/3000
2/2 [==============================] - 0s 2ms/step - loss: 0.5745
Epoch 112/3000
2/2 [==============================] - 0s 3ms/step - loss: 0.5731
Epoch 113/3000
2/2 [==============================] - 0s 3ms/step - loss: 0.5717
Epoch 114/3000
2/2 [==============================] - 0s 2ms/step - loss: 0.5704
Epoch 115/3000
2/2 [=====================

2/2 [==============================] - 0s 3ms/step - loss: 0.4971
Epoch 205/3000
2/2 [==============================] - 0s 3ms/step - loss: 0.4966
Epoch 206/3000
2/2 [==============================] - 0s 4ms/step - loss: 0.4961
Epoch 207/3000
2/2 [==============================] - 0s 2ms/step - loss: 0.4956
Epoch 208/3000
2/2 [==============================] - 0s 3ms/step - loss: 0.4951
Epoch 209/3000
2/2 [==============================] - 0s 2ms/step - loss: 0.4947
Epoch 210/3000
2/2 [==============================] - 0s 2ms/step - loss: 0.4942
Epoch 211/3000
2/2 [==============================] - 0s 2ms/step - loss: 0.4939
Epoch 212/3000
2/2 [==============================] - 0s 1ms/step - loss: 0.4935
Epoch 213/3000
2/2 [==============================] - 0s 3ms/step - loss: 0.4931
Epoch 214/3000
2/2 [==============================] - 0s 2ms/step - loss: 0.4927
Epoch 215/3000
2/2 [==============================] - 0s 2ms/step - loss: 0.4923
Epoch 216/3000
2/2 [=======================

2/2 [==============================] - 0s 4ms/step - loss: 0.4597
Epoch 306/3000
2/2 [==============================] - 0s 4ms/step - loss: 0.4594
Epoch 307/3000
2/2 [==============================] - 0s 4ms/step - loss: 0.4591
Epoch 308/3000
2/2 [==============================] - 0s 3ms/step - loss: 0.4589
Epoch 309/3000
2/2 [==============================] - 0s 3ms/step - loss: 0.4586
Epoch 310/3000
2/2 [==============================] - 0s 3ms/step - loss: 0.4583
Epoch 311/3000
2/2 [==============================] - 0s 4ms/step - loss: 0.4580
Epoch 312/3000
2/2 [==============================] - 0s 3ms/step - loss: 0.4577
Epoch 313/3000
2/2 [==============================] - 0s 4ms/step - loss: 0.4574
Epoch 314/3000
2/2 [==============================] - 0s 3ms/step - loss: 0.4571
Epoch 315/3000
2/2 [==============================] - 0s 2ms/step - loss: 0.4567
Epoch 316/3000
2/2 [==============================] - 0s 1ms/step - loss: 0.4564
Epoch 317/3000
2/2 [=======================

2/2 [==============================] - 0s 3ms/step - loss: 0.4374
Epoch 407/3000
2/2 [==============================] - 0s 2ms/step - loss: 0.4373
Epoch 408/3000
2/2 [==============================] - 0s 3ms/step - loss: 0.4371
Epoch 409/3000
2/2 [==============================] - 0s 2ms/step - loss: 0.4370
Epoch 410/3000
2/2 [==============================] - 0s 2ms/step - loss: 0.4369
Epoch 411/3000
2/2 [==============================] - 0s 2ms/step - loss: 0.4367
Epoch 412/3000
2/2 [==============================] - 0s 3ms/step - loss: 0.4366
Epoch 413/3000
2/2 [==============================] - 0s 3ms/step - loss: 0.4364
Epoch 414/3000
2/2 [==============================] - 0s 4ms/step - loss: 0.4362
Epoch 415/3000
2/2 [==============================] - 0s 5ms/step - loss: 0.4360
Epoch 416/3000
2/2 [==============================] - 0s 3ms/step - loss: 0.4359
Epoch 417/3000
2/2 [==============================] - 0s 2ms/step - loss: 0.4357
Epoch 418/3000
2/2 [=======================

...

2/2 [==============================] - 0s 3ms/step - loss: 0.3855
Epoch 2507/3000
2/2 [==============================] - 0s 2ms/step - loss: 0.3855
Epoch 2508/3000
2/2 [==============================] - 0s 4ms/step - loss: 0.3855
Epoch 2509/3000
2/2 [==============================] - 0s 4ms/step - loss: 0.3855
Epoch 2510/3000
2/2 [==============================] - 0s 2ms/step - loss: 0.3855
Epoch 2511/3000
2/2 [==============================] - 0s 2ms/step - loss: 0.3855
Epoch 2512/3000
2/2 [==============================] - 0s 3ms/step - loss: 0.3855
Epoch 2513/3000
2/2 [==============================] - 0s 3ms/step - loss: 0.3855
Epoch 2514/3000
2/2 [==============================] - 0s 2ms/step - loss: 0.3855
Epoch 2515/3000
2/2 [==============================] - 0s 2ms/step - loss: 0.3855
Epoch 2516/3000
2/2 [==============================] - 0s 2ms/step - loss: 0.3855
Epoch 2517/3000
2/2 [==============================] - 0s 2ms/step - loss: 0.3855
Epoch 2518/3000
2/2 [===========

2/2 [==============================] - 0s 3ms/step - loss: 0.3853
Epoch 2607/3000
2/2 [==============================] - 0s 3ms/step - loss: 0.3853
Epoch 2608/3000
2/2 [==============================] - 0s 3ms/step - loss: 0.3853
Epoch 2609/3000
2/2 [==============================] - 0s 3ms/step - loss: 0.3853
Epoch 2610/3000
2/2 [==============================] - 0s 3ms/step - loss: 0.3853
Epoch 2611/3000
2/2 [==============================] - 0s 4ms/step - loss: 0.3853
Epoch 2612/3000
2/2 [==============================] - 0s 3ms/step - loss: 0.3853
Epoch 2613/3000
2/2 [==============================] - 0s 3ms/step - loss: 0.3853
Epoch 2614/3000
2/2 [==============================] - 0s 2ms/step - loss: 0.3853
Epoch 2615/3000
2/2 [==============================] - 0s 2ms/step - loss: 0.3853
Epoch 2616/3000
2/2 [==============================] - 0s 2ms/step - loss: 0.3853
Epoch 2617/3000
2/2 [==============================] - 0s 3ms/step - loss: 0.3853
Epoch 2618/3000
2/2 [===========

2/2 [==============================] - 0s 2ms/step - loss: 0.3852
Epoch 2707/3000
2/2 [==============================] - 0s 3ms/step - loss: 0.3852
Epoch 2708/3000
2/2 [==============================] - 0s 998us/step - loss: 0.3852
Epoch 2709/3000
2/2 [==============================] - 0s 3ms/step - loss: 0.3852
Epoch 2710/3000
2/2 [==============================] - 0s 2ms/step - loss: 0.3852
Epoch 2711/3000
2/2 [==============================] - 0s 1ms/step - loss: 0.3852
Epoch 2712/3000
2/2 [==============================] - 0s 3ms/step - loss: 0.3852
Epoch 2713/3000
2/2 [==============================] - 0s 2ms/step - loss: 0.3852
Epoch 2714/3000
2/2 [==============================] - 0s 998us/step - loss: 0.3852
Epoch 2715/3000
2/2 [==============================] - 0s 2ms/step - loss: 0.3852
Epoch 2716/3000
2/2 [==============================] - 0s 2ms/step - loss: 0.3852
Epoch 2717/3000
2/2 [==============================] - 0s 3ms/step - loss: 0.3852
Epoch 2718/3000
2/2 [=======

Epoch 2806/3000
2/2 [==============================] - 0s 3ms/step - loss: 0.3851
Epoch 2807/3000
2/2 [==============================] - 0s 2ms/step - loss: 0.3851
Epoch 2808/3000
2/2 [==============================] - 0s 2ms/step - loss: 0.3851
Epoch 2809/3000
2/2 [==============================] - 0s 2ms/step - loss: 0.3851
Epoch 2810/3000
2/2 [==============================] - 0s 3ms/step - loss: 0.3851
Epoch 2811/3000
2/2 [==============================] - 0s 3ms/step - loss: 0.3850
Epoch 2812/3000
2/2 [==============================] - 0s 4ms/step - loss: 0.3850
Epoch 2813/3000
2/2 [==============================] - 0s 2ms/step - loss: 0.3850
Epoch 2814/3000
2/2 [==============================] - 0s 3ms/step - loss: 0.3850
Epoch 2815/3000
2/2 [==============================] - 0s 2ms/step - loss: 0.3850
Epoch 2816/3000
2/2 [==============================] - 0s 3ms/step - loss: 0.3850
Epoch 2817/3000
2/2 [==============================] - 0s 4ms/step - loss: 0.3850
Epoch 2818/3000


2/2 [==============================] - 0s 2ms/step - loss: 0.3849
Epoch 2906/3000
2/2 [==============================] - 0s 2ms/step - loss: 0.3849
Epoch 2907/3000
2/2 [==============================] - 0s 3ms/step - loss: 0.3849
Epoch 2908/3000
2/2 [==============================] - 0s 2ms/step - loss: 0.3849
Epoch 2909/3000
2/2 [==============================] - 0s 2ms/step - loss: 0.3849
Epoch 2910/3000
2/2 [==============================] - 0s 3ms/step - loss: 0.3849
Epoch 2911/3000
2/2 [==============================] - 0s 2ms/step - loss: 0.3849
Epoch 2912/3000
2/2 [==============================] - 0s 2ms/step - loss: 0.3849
Epoch 2913/3000
2/2 [==============================] - 0s 2ms/step - loss: 0.3849
Epoch 2914/3000
2/2 [==============================] - 0s 2ms/step - loss: 0.3849
Epoch 2915/3000
2/2 [==============================] - 0s 2ms/step - loss: 0.3849
Epoch 2916/3000
2/2 [==============================] - 0s 3ms/step - loss: 0.3849
Epoch 2917/3000
2/2 [===========

In [11]:
ae_kc_2.save("ae_kc_2")

INFO:tensorflow:Assets written to: ae_kc_2\assets


In [12]:
H_kc_2 = ae_kc_2.encoder(H_kc_1)

In [13]:
kmeans = KMeans(n_clusters=10, random_state=0).fit(H_kc_2)

In [ ]:
labels = karate_club_communities(kc)

In [14]:
normalized_mutual_info_score(labels, kmeans.labels_)

0.4493583818010334

In [15]:
normalized_mutual_info_score([6,1,4,1,3,2,6,4,3,5,6,2], [3,1,6,3,4,1,2,1,4,2,3,1])

0.6902532446600738

In [16]:
labels

NameError: name 'labels' is not defined